## Exploration of 100d space of genome vectors

Genome vectors created by the Dna2VecDataBunch exhibit piculiar patterns. This notebook is dedicated to exploratoin 
of the bacterial genome space using dimensionality reduction techniques

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../mylib/")

from genomic import sequence
from genomic.sequence import regex_filter, count_filter
from functools import partial
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn import manifold,neighbors
from scipy.cluster.hierarchy import dendrogram, linkage  
from matplotlib import pyplot as plt
import seaborn as sns; sns.set(color_codes=True)


### Load Data

In [3]:
filters=[partial(regex_filter, rx="Streptomyces|Bacillus|Vibrio|Streptococcus|Rhizobium|Staphylococcus"),partial(regex_filter, rx="plasmid", keep=False),
         partial(count_filter, max_count=599)]
data = sequence.Dna2VecList.from_folder("/data/genomes/GenSeq_fastas/train",filters=filters,agg=partial(np.mean, axis=0),n_cpus=7)
processors = [
    sequence.GSFileProcessor(),
    sequence.GSTokenizeProcessor(tokenizer=sequence.GSTokenizer(ngram=8, skip=0, n_cpus=7)),
    sequence.Dna2VecProcessor()]
%time for p in processors: p.process(data)

CPU times: user 47.7 s, sys: 1.65 s, total: 49.3 s
Wall time: 51.9 s


In [4]:
len(data.items)

3169

###  Genome vectors

In [54]:
def log_scale(X):
    x=np.asarray(X);e=1e-6
    return np.log10(x+np.abs(x.min())+e) 


x=np.asarray(data.items)
bad_fastas = np.where(np.mean(x,axis=1) == 0.)[0]
X = np.delete(x, bad_fastas,0)
labelList=[" ".join(i.split()[1:3]) for i in data.descriptions]
labelList=np.delete(np.asarray(labelList), bad_fastas)
vocab=list(np.unique(labelList))
y=[vocab.index(x) for x in labelList]

## tSNE

In [91]:
tsne = manifold.TSNE(n_components=3, init='pca', perplexity=10, metric="correlation",random_state=0)
%time X3 = tsne.fit_transform(log_scale(X))

CPU times: user 1min 51s, sys: 363 ms, total: 1min 52s
Wall time: 1min 51s


In [114]:
genus = [i.split()[0] for i in labelList]
genus_vocab=list(np.unique(genus))
y=[genus_vocab.index(x) for x in genus]
genus_vocab

['Bacillus',
 'Rhizobium',
 'Staphylococcus',
 'Streptococcus',
 'Streptomyces',
 'Vibrio']

In [103]:
X3_df = pd.DataFrame(data=X3, columns=["pc1",'pc2','pc3'], index=labelList)

X3_df["genus"]=genus
X3_df["y"]=y

### Plotly visualisation

In [107]:
import plotly.plotly as py
import plotly.graph_objs as go

In [113]:
genomes = go.Scatter3d(
    x=X3_df.pc1,
    y=X3_df.pc2,
    z=X3_df.pc3,
    mode='markers',
    marker=dict(
        size=8,
        color=X3_df.y+1,                # set color to an array/list of desired values
        colorscale='Viridis',           # choose a colorscale
        opacity=0.5
    )
)

data = [genomes]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='bacterial_genome embedding space')

/home/serge/anaconda3/envs/bio/lib/python3.6/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



## Genome Inventory

In [115]:
all_fastas = sequence.Dna2VecList.from_folder("/data/genomes/GenSeq_fastas/train").descriptions
inventory = pd.DataFrame(data=[l.split()[1:3] for l in all_fastas], columns=["genus","species" ]).groupby(["genus", "species"]).agg({"species": "count"})
inventory.columns=["count"]
inventory

count
genus            species                      
Bacillus         cereus                      2
                 coahuilensis              138
                 eiseniae                    9
                 krulwichiae                 1
                 mannanilyticus            108
                 massilioanorexius         120
                 panaciterrae              189
                 psychrosaccharolyticus    265
                 solani                     12
                 subterraneus              208
                 vallismortis               80
Clostridium      aceticum                    2
                 amylolyticum               20
                 butyricum                   2
                 chauvoei                    1
                 haemolyticum              234
Corynebacterium  accolens                   38
                 atypicum                    2
                 doosanense                  2
                 genitalium                  1
                 glucuronolyticum           41
                 imitans                     1
                 timonense                  18
Enterobacter     aerogenes                   1
Escherichia      albertii                    1
                 fergusonii                  2
                 marmotae                  130
Klebsiella       oxytoca                    12
                 quasipneumoniae             3
                 variicola                   1
...                                        ...
Shigella         boydii                      6
                 dysenteriae                 3
                 sonnei                      5
Staphylococcus   agnetis                     2
                 aureus                      1
                 hyicus                      1
                 intermedius               286
                 warneri                     9
                 xylosus                     2
Stenotrophomonas acidaminiphila              1
                 ginsengisoli               99
                 koreensis                  58
                 pavanii                    17
                 rhizophila                  1
Streptococcus    acidominimus              111
                 azizii                     43
                 entericus                  33
                 himalayensis                1
                 minor                      33
                 oralis                      1
Streptomyces     albidoflavus              115
                 ambofaciens                 2
                 antibioticus              343
                 antioxidans               282
                 filamentosus                1
Vibrio           aerogenes                  77
                 celticus                  196
                 europaeus                  10
                 ruber                     103
                 xiamenensis                82

[88 rows x 1 columns]

In [117]:
counts = inventory.reset_index().groupby("genus").agg({"count", sum}).drop(("species"), axis=1)
counts.columns=["n_sequences","species"]
counts.sort_values("n_sequences", ascending=False)

,n_sequences,species
genus,,
Bacillus,1132,11
Streptomyces,743,5
Vibrio,468,5
Rhizobium,325,6
Pseudomonas,304,8
Staphylococcus,301,6
Clostridium,259,5
Streptococcus,222,6
Planktothrix,179,5
